#  Azure Text Classification - preprocessed vs unprocessed data

In [3]:
import os
import pandas as pd
import numpy as np

Below I loop through the text files in the data folder, create a dictionary with the file's text and name, and then save the dictionaries in a list named `text_data`. The list is then convereted into a dataframe and saved as data.csv.

In [4]:
text_data = []

for root, directories, files in os.walk("data"):
    for filename in files:
        if filename.endswith(".txt"):
            text_data.append({"text": open(f"data/{filename}").read(), "filename": filename})

In [5]:
df = pd.DataFrame(text_data)
df.to_csv('data.csv', index=False)

In [6]:
df = pd.read_csv('data.csv')

In [33]:
df.head(5)

,text,filename,stopwords,punctuation,numerics,upper
0,versatile potentiostat base inexpensive shelf ...,060.txt,42,12,0,1
1,introduction family play important role patien...,074.txt,84,27,1,2
2,novel hybrid compose photocatalytic fuel pfc f...,048.txt,40,23,2,12
3,einsteinaether theory violate lorentz invarian...,114.txt,45,27,0,0
4,vigorous spread renewable energy much attentio...,100.txt,72,26,1,2


## Feature Extraction and Preprocessing 

The following feature extraction and preprocessing code is based on the work found here: https://github.com/EnesGokceDS/Amazon_Reviews_NLP_Capstone_Project

In [8]:
import wordcloud
from nltk.corpus import stopwords
import nltk
import string
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/patmellon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/patmellon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/patmellon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/patmellon/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
# Count the stopwords
df['stopwords'] = df['text'].apply(lambda x: len([x for x in x.split() if x in stop]))
df[['text','stopwords']].head()

,text,stopwords
0,A versatile potentiostat based on inexpensive ...,42
1,Introduction: Family plays an important role i...,84
2,A novel hybrid system composed of a photocatal...,40
3,"In Einstein-aether theory, violating Lorentz i...",45
4,"With the vigorous spread of renewable energy, ...",72


In [10]:
# Count the punctuation
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return count

df['punctuation'] = df['text'].apply(lambda x: count_punct(x))

In [11]:
df[['text','punctuation']].head()

,text,punctuation
0,A versatile potentiostat based on inexpensive ...,12
1,Introduction: Family plays an important role i...,27
2,A novel hybrid system composed of a photocatal...,23
3,"In Einstein-aether theory, violating Lorentz i...",27
4,"With the vigorous spread of renewable energy, ...",26


In [12]:
# Count the numbers
df['numerics'] = df['text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
df[['text','numerics']].head()

,text,numerics
0,A versatile potentiostat based on inexpensive ...,0
1,Introduction: Family plays an important role i...,1
2,A novel hybrid system composed of a photocatal...,2
3,"In Einstein-aether theory, violating Lorentz i...",0
4,"With the vigorous spread of renewable energy, ...",1


In [13]:
# Count the uppercase words
df['upper'] = df['text'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
df[['text','upper']].head()

,text,upper
0,A versatile potentiostat based on inexpensive ...,1
1,Introduction: Family plays an important role i...,2
2,A novel hybrid system composed of a photocatal...,12
3,"In Einstein-aether theory, violating Lorentz i...",0
4,"With the vigorous spread of renewable energy, ...",2


#### Preprocess Data

In [35]:
# Convert text to lowercase
df['text'] = df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['text'].head()

0    versatile potentiostat base inexpensive shelf ...
1    introduction family play important role patien...
2    novel hybrid compose photocatalytic fuel pfc f...
3    einsteinaether theory violate lorentz invarian...
4    vigorous spread renewable energy much attentio...
Name: text, dtype: object

In [36]:
# Remove punctuation
df['text'] = df['text'].str.replace('[^\w\s]','')
df['text'].head()

/var/folders/fx/wfh1hq7s74l7s32dj3t81jdc0000gn/T/ipykernel_31686/3558066172.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('[^\w\s]','')


0    versatile potentiostat base inexpensive shelf ...
1    introduction family play important role patien...
2    novel hybrid compose photocatalytic fuel pfc f...
3    einsteinaether theory violate lorentz invarian...
4    vigorous spread renewable energy much attentio...
Name: text, dtype: object

In [37]:
# Remove stop words
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['text'].sample(10)

184    compact ghz doppler radar module develop paper...
26     analytic thermodynamic establish quantitativel...
138    melatonin affect variety physiological process...
91     background erythropoiesisstimulating agent esa...
115    brief overview vortexinduced vibration viv bri...
207    physician implicit racial bias find predict ra...
36     propose calculate composition gaasxpx solid so...
117    vi european guideline cardiovascular preventio...
175    project owner important project participant in...
164    green roof propose efficient practical tool co...
Name: text, dtype: object

In [38]:
# Correct spelling
from textblob import TextBlob
df['text'][:5].apply(lambda x: str(TextBlob(x).correct()))

0    versatile potentiostat base expensive shelf co...
1    introduction family play important role patien...
2    novel horrid compose photocatalytic fuel of fe...
3    einsteinaether theory violate lorentz variance...
4    vigorous spread renewal energy much attention ...
Name: text, dtype: object

In [39]:
# Save the preprocessed_data as a new CSV
df.to_csv('processed_data.csv', index=False)

### Upload data to Azure

The below code is based on this Quickstart guide: https://docs.microsoft.com/en-us/azure/storage/blobs/storage-quickstart-blobs-python?tabs=environment-variable-linux.

In [20]:
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv

load_dotenv()

AZURE_STORAGE_CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
CONTAINER_NAME = os.getenv("CONTAINER_NAME")

blob_service_client = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)

try:
    print("Uploading...")
    
    # Loop through each row and upload to Azure
    for _index, row in df.iterrows():
        blob_client = blob_service_client.get_blob_client(container=CONTAINER_NAME, blob=row['filename'])
        # Set overwrite to true 
        blob_client.upload_blob(row['text'], overwrite=True)
    
    print("Finished")

except Exception as ex:
    print('Exception:')
    print(ex)

Azure Blob Storage v12.13.0 - Python quickstart sample


In [ ]:
# Remove numbers, lemmatization, and whitespace

In [40]:
# Remove numbers
df['text'] = df['text'].str.replace('\d+', '')
df['text'].head()

/var/folders/fx/wfh1hq7s74l7s32dj3t81jdc0000gn/T/ipykernel_31686/1416691737.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('\d+', '')


0    versatile potentiostat base inexpensive shelf ...
1    introduction family play important role patien...
2    novel hybrid compose photocatalytic fuel pfc f...
3    einsteinaether theory violate lorentz invarian...
4    vigorous spread renewable energy much attentio...
Name: text, dtype: object

In [23]:
df.to_csv('processed_data.csv', index=False)

In [41]:
# https://www.kaggle.com/code/sudalairajkumar/getting-started-with-text-preprocessing
import nltk
nltk.download('omw-1.4')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

df["text"] = df["text"].apply(lambda text: lemmatize_words(text))
df.head()

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/patmellon/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,text,filename,stopwords,punctuation,numerics,upper
0,versatile potentiostat base inexpensive shelf ...,060.txt,42,12,0,1
1,introduction family play important role patien...,074.txt,84,27,1,2
2,novel hybrid compose photocatalytic fuel pfc f...,048.txt,40,23,2,12
3,einsteinaether theory violate lorentz invarian...,114.txt,45,27,0,0
4,vigorous spread renewable energy much attentio...,100.txt,72,26,1,2


In [47]:
# Remove whitespace
df["text"] = df["text"].str.strip() 

In [48]:
df.head()

,text,filename,stopwords,punctuation,numerics,upper
0,versatile potentiostat base inexpensive shelf ...,060.txt,42,12,0,1
1,introduction family play important role patien...,074.txt,84,27,1,2
2,novel hybrid compose photocatalytic fuel pfc f...,048.txt,40,23,2,12
3,einsteinaether theory violate lorentz invarian...,114.txt,45,27,0,0
4,vigorous spread renewable energy much attentio...,100.txt,72,26,1,2


In [49]:
df.to_csv('processed_data.csv', index=False)

In [50]:
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv

load_dotenv()

AZURE_STORAGE_CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
CONTAINER_NAME = os.getenv("CONTAINER_NAME")

blob_service_client = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)

try:
    print("Uploading...")
    
    # Loop through each row and upload to Azure
    for _index, row in df.iterrows():
        blob_client = blob_service_client.get_blob_client(container=CONTAINER_NAME, blob=row['filename'])
        # Set overwrite to true 
        blob_client.upload_blob(row['text'], overwrite=True)
    
    print("Finished")

except Exception as ex:
    print('Exception:')
    print(ex)

Uploading...
Finished
